In [186]:
import numpy as np
import random

class NN():
    """
    Class for a neural network. Requires input/output sizes, number of hidden layers, and number of neurons
    at each layer (we assume all hidden layers are of the same size)
    """
    def __init__(self, input_size, num_HL, hidden_size, output_size):
        # Initialize by setting random 
        self.input_size = input_size
        self.hidden_layers = num_HL
        # NOTE we are assuming all hidden layers are the same size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Activations for each neuron
        # + 1 for bias neuron
        self.activations_in = np.ones(self.input_size + 1)
        # Hidden can comprise multiple layers, so we have a matrix
        # + 1 for bias neuron
        self.activations_hidden = np.ones((self.hidden_layers, self.hidden_size + 1))
        self.activations_out = np.ones(self.output_size)
        # Weights of all the edges, randomized for good results
        # PLUS ONE FOR BIASES
        self.weights_in = np.random.randn(self.input_size + 1, self.hidden_size)
        self.weights_in = np.column_stack((self.weights_in, np.zeros(self.input_size + 1)))
        # We will only have hidden weights if there are multiple hidden layers
        if self.hidden_layers > 1:
            self.weights_hidden = np.random.randn(self.hidden_layers - 1, self.hidden_size + 1, self.hidden_size + 1)
            # Set the weights corresponding with next layers biases to 0
            for weights in self.weights_hidden:
                weights[-1] = 0.0
        else:
            self.weights_hidden = []
        # No plus one for output, as it should not have a bias parameter
        self.weights_out = np.random.randn(self.hidden_size + 1, self.output_size)
        # To be valued when train() is called
        self.learning_rate = 0.0

        # Instantiate deltas for holding gradients
        self.deltas_in = []
        self.Deltas_hidden = []
        self.deltas_out = []
    
    def _sigmoid(self, x):
        """
        Sigmoid function for calculating a distribution over 2 classes
        """
        return 1 / (1 + np.exp(-x))
    
    def _derivative_sigmoid(self, x):
        """
        Derivative of the sigmoid function where x = the output of the sigmoid
        
        This can be used in backpropogation, wherein we would have 
        already computed the sigmoid in the forward pass, and we can draw upon its cached value
        """
        return self._sigmoid(x) * (1.0 - self._sigmoid(x))
    
    def _softmax(self, x):
        exponentials = [np.exp(p) for p in x]
        denominator = sum(exponentials)
        return [p / denominator for p in exponentials]
    
    def _relu(self, x):
        """
        relu function used for activation
        """
        return max(x, 0.0)
    
    def _derivative_relu(self, x):
        """
        Derivative of the relu function, the input will be the output of the relu function.
        This is because in practice we will have already performed this computation in the forward pass
        so in the backward pass, we need to find its derivative drawing upon the cached relu(x).
        """
        return 1 if x > 0.0 else 0.0
    
    def _binary_cross_ent(self, y_hat, y):
        """
        This basically finds the negative of the log probability of class1 - its inverse
        """
        return (-y * np.log(y_hat)) - ((1 - y) * np.log(1 - y_hat))
    
    def _negative_log_likelihood(self, y_hat):
        return -np.log(y_hat)
    
    def _derivative_negative_log_likelihood(self, y_hat):
        return -1/y_hat
    
    def _derivative_binary_cross_ent(self, y_hat, y):
        """
        Derivative of binary cross-entropy
        
        This description is misleading. 
        This is the part of the partial derivative of binary cross-entropy 
        w.r.t the parameters of our function. In practice, the other part is 
        the dot product of this and the activations (activate(w, x))
        """
        #return -(y / y_hat) - ((1 - y) / (1 - y_hat))
        return (y_hat - y)

    def _activate(self, x):
        """
        RELU for non-linear activation function
        """
        return self._relu(x)
    
    def _activate_vector(self, X):
        """
        Run on a numpy vector
        """
        activations = np.vectorize(self._activate)
        return activations(X)
    
    def _derivative_activation(self, x):
        """
        Compute the derivative of the activation function given the activation output
        
        x: activate(node)
        """
        return self._derivative_relu(x)
    
    def _derivative_vector_activation(self, X):
        """
        Derivative for each scalar in a numpy vector
        """
        derivative_activations = np.vectorize(self._derivative_activation)
        return derivative_activations(X)

    def _loss(self, y_hat, y):
        """
        y_hat: sofmax vector
        y:     one-hot vector for the target
        
        Here we will plug in the negative_log_likelihood
        in order to be able to compare the proability of our output at
        the correct class, to 1.
        """
        
        """
        Get the index of the correct class 
        (numpy will return a tuple of the index in each dimension)
        """
        
        # Get the first (and only) dim, and the first (and only) index
        i = np.where(y==1)[0][0]
        return self._negative_log_likelihood(y_hat[i])
    
    def _derivative_loss(self, y_hat, y):
        """
        This will be used in backprop for finding L'(output_layer_node)
        """
        # Get the first (and only) dim, and the first (and only) index
        i = np.where(y==1)[0][0]
        return self._derivative_negative_log_likelihood(y_hat[i])
    
    def _targets_to_one_hots(self, targets):
        """
        Interpret a vector of targets into a matrix
        of one-hot representations
        """
        # Get the number of unique target classes
        num_classes = len(set(targets))
        # Instantiate a matrix of one-hot vectors
        # with one row per target, and one col per class
        one_hots = np.zeros((len(targets), num_classes))
        for i, one_hot in enumerate(one_hots):
            # Set the one-hot vector to hae a 1 at its corresponding target slot
            t = targets[i]
            one_hot[t] = 1
            
        return one_hots
        
    def forward(self, inputs):
        """
        Forward pass: Calculate the activations of each neuron
        """
        if len(inputs) != self.input_size:
          raise Exception("That is not the size of the input layer... try %i" % self.input_size)
        
        # Set input activations, no need to actually calculate anything
        for i, input in enumerate(inputs):
            self.activations_in[i] = input
        
        # calculate the activations for each hidden layer
        for h_layer_i in range(self.hidden_layers):
            # Need to take previous layer activation value * weights for a given layer
            # Starting with input layer X first hidden layer
            if h_layer_i == 0:
                """
                # Loop over the layer
                for h_dim_j in range(self.hidden_size):
                    # Loop over neurons in the layer before it
                    for k in range(self.input_size):
                        # Sum [f_k * w_k_j for k in input layer]
                        self.activations_hidden[h_layer_i][h_dim_j] += self.activations_in[k] * self.weights_in[k][h_dim_j]
                    # h(sum from above), aka run the nonlinear activation function
                    self.activations_hidden[h_layer_i][h_dim_j] = self._activate(self.activations_hidden[h_layer_i][h_dim_j])
                """
                # multiply the previous layer's activations by its weight vector for this layer's activations
                self.activations_hidden[h_layer_i] = self.activations_in.T.dot(self.weights_in)
                # Reset bias activation to 1.0
                self.activations_hidden[h_layer_i][-1] = 1.0
            else:
                """
                if h_layer_i == 0:
                    # Loop over the layer
                    for h_dim_j in range(self.hidden_size):
                        # Loop over neurons in the layer before it
                        for k in range(self.hidden_size):
                            # Sum [f_k * w_k_j for k in previous hidden layer]
                            self.activations_hidden[h_layer_i][h_dim_j] += self.activations_hidden[h_layer_i - 1][k]\
                            * self.weights_hidden[h_layer_i][k][h_dim_j]
                        # h(sum from above), aka run the nonlinear activation function
                        self.activations_hidden[h_layer_i][h_dim_j] = self.activations_hidden[h_layer_i][h_dim_j]
                """
                # multiply the previous layer's activations by its weight vector for this layer's activations
                self.activations_hidden[h_layer_i] = self._activate_vector(self.activations_hidden[h_layer_i - 1]).T.dot(self.weights_hidden[h_layer_i - 1])
                # Reset bias activation to 1.0
                self.activations_hidden[h_layer_i][-1] = 1.0

                
        # Output activations will be the dot product of the final hidden layer, and the output weights
        """
        for i in range(self.output_size):
            for j in range(self.hidden_size):
                print(self.weights_out)
                print(self.weights_out[j][i])
                self.activations_out[i] += self.activations_hidden[-1][j] * self.weights_out[j][i]
            # h(sum from above), aka run the nonlinear activation function
            self.activations_out[i] = self._activate(self.activations_out[i])
        """
        # Activate the vector before, but do not activate the activations_out
        self.activations_out = self._activate(self.activations_hidden[-1]).T.dot(self.weights_out)
        # [:-1] because this one should not have a bias parameter
        #self.activations_out = self._activate_vector(self.activations_out)
        
        #Print all of the weights, to see updates
        """
        print("ACTIVATIONS")
        print(self.activations_in)
        print(self.activations_hidden)
        print(self.activations_out)
        """
        print("WEIGHTS:")
        print(self.weights_in)
        print(self.weights_hidden)
        print(self.weights_out)

    def backward(self, targets):
        """
        Backpropogation for finding the partial derivative of the each node w.r.t the loss function,
        and updating weights based on those gradients
        """
        if len(targets) != len(self.activations_out):
            raise Exception("Your labels are not the same size as your output layer!")
        # Calculate loss - there will be a value for each node in the output layer
        # Take the simoid of the activations of the output layer, because we are doing 2 class classification
        # ***If we have >2 classes, we would use softmax***
        """
        print("ACTIVATIONS IN")
        print(self.activations_in)
        
        print("ACTIVATIONS HIDDEN")
        print(self.activations_hidden)
        
        print("ACTIVATIONS OUT")
        print(self.activations_out)
        """
        
        print("SOFTMAX_LAYER")
        print(self._softmax(self.activations_out))
        """
        print("TARGETS")
        print(targets)
        """
        loss = self._loss(self._softmax(self.activations_out), targets)
        
        """
        Now we need to calculate the partial derivative of the loss w.r.t each weight.
        Think of this as finding the amount that each node contributes to a change in the final loss.
        
        Each node has a value "delta", which represents the partial derivative of the loss w.r.t. its value:
        Use the partial derivative of the loss function, in our case binary cross-entropy
        """
        self.deltas_out = np.zeros([self.output_size])
        derivative_loss = self._derivative_loss(self._softmax(self.activations_out), targets)
        print("LOSS PER INSTANCE: %2f" % loss)
        print("DERIVATIVE LOSS: %2f" % derivative_loss)
        print("DERIVATIVE ACTIVATIONS:")
        print(self._derivative_vector_activation(self.activations_out))
        for i, activation_out in enumerate(self.activations_out):
            self.deltas_out[i] = derivative_loss * self._derivative_activation(activation_out)
        
        """
        Find derivative of activation (activation was found in the forward pass) * derivative of the inner function,
        which is the parameter w
        """
        self.Deltas_hidden = np.zeros([self.hidden_layers, self.hidden_size + 1])
        ##############
        #####TODO Needs to go in reverse if we have multiple hidden
        for h_layer_i in range(self.hidden_layers):
            # If it is the last hidden layer, then we look at the activations and deltas
            # of the output layer, not the next hidden layer
            print("LAYER: %i" % h_layer_i)
            if h_layer_i == self.hidden_layers - 1:
                # Loop over each in hidden activation, +1 for bias
                for h_dim_j in range(self.hidden_size + 1):
                    for k, delta_out in enumerate(self.deltas_out):
                        self.Deltas_hidden[h_layer_i][h_dim_j] += delta_out * self._derivative_activation(self.activations_out[k]) * self.weights_out[h_dim_j][k]
            else:
                # Do the same to find the hidden deltas
                for h_dim_j in range(self.hidden_size + 1):
                    for k, delta_h in enumerate(self.Deltas_hidden[h_layer_i + 1]):
                        self.Deltas_hidden[h_layer_i][h_dim_j] += delta_h * self._derivative_activation(self.activations_hidden[h_layer_i + 1][k]) * self.weights_hidden[h_layer_i][h_dim_j][k]
                        
        """
        Now just do the same for L'(input layer)
        """
        self.deltas_in = np.zeros([self.input_size+1])
        #### Need deltas_hidden[0].dot(der_act_hidden[0] * weights_in)
        # Loop over each in input activation, +1 for bias
        #for dim_i in range(self.input_size + 1):
        #    for k, delta_h in enumerate(self.Deltas_hidden[0]):
        #        self.deltas_in[dim_i] += delta_h * self._derivative_activation(self.activations_hidden[0][k]) * self.weights_in[dim_i][k]
        
        self.update_weights()
        return loss
        
    def update_weights(self):
        print("UPDATING WEIGHTS")
        # Now we can use the deltas to adjust each weight by L'(w_i_j)
        # These weights are the edges shared between last hidden layer, and output layer
        # Rows of weights_out correponds with length of last hidden layer
        for i in range(len(self.weights_out)):
            # Cols of weights_out correspinds with length of output layer
            for j in range(len(self.weights_out[i])):
                self.weights_out[i][j] -= self._activate(self.activations_hidden[-1][i])\
                * self._derivative_activation(self.activations_out[j])\
                * self.deltas_out[j]\
                * self.learning_rate
                    
        # Loop over each hidden layer
        for w_i in reversed(range(len(self.weights_hidden))):
            # Rows (i) in the weights for this layer will correspond to the size of the layer BEFORE (hidden or input)
            for i in range(len(self.weights_hidden[w_i])):
                # Cols (j) in these weights will correspond to the size of hidden layer w_i
                for j in range(len(self.weights_hidden[w_i][i])):
                    # We do not want to update the dummy 0 weight going TO the extra bias dimension
                    if j == len(self.weights_hidden[w_i][i]) - 1:
                        continue
                    else:
                        # + 1 for layer before, because we are looping in reverse
                        self.weights_hidden[w_i][i][j] -= self._activate(self.activations_hidden[w_i + 1][i])\
                        * self._derivative_activation(self.activations_hidden[w_i][j])\
                        * self.Deltas_hidden[w_i][j]\
                        * self.learning_rate
                        
            #self.weights_hidden[w_i] -= self.activations_hidden[w_i].T.dot(self.Deltas_hidden[w_i]) * self.learning_rate
        # Rows (i) of weights_in corresponds to size of the input layer
        for i in range(len(self.weights_in)):
            # Cols (j) corresponds to size of the first hidden layer (layer above input layer)
            for j in range(len(self.weights_in[i])):
                # We do not want to update the dummy 0 weight going TO the extra bias dimension
                if j == len(self.weights_in[i]) - 1:
                    continue
                else:
                    """
                    print("act, deriv_act, delta, weight update val")
                    print(self.activations_in[i])
                    print(self._derivative_activation(self.activations_hidden[0][j]))
                    print(self.Deltas_hidden[0][j])
                    print(self.activations_in[i]\
                    * self._derivative_activation(self.activations_hidden[0][j])\
                    * self.Deltas_hidden[0][j]\
                    * self.learning_rate)
                    """
                    
                    self.weights_in[i][j] -= self.activations_in[i]\
                    * self._derivative_activation(self.activations_hidden[0][j])\
                    * self.Deltas_hidden[0][j]\
                    * self.learning_rate
    
    def train(self, inputs, targets, epochs=50, lr=.01):
        self.learning_rate = lr
        one_hot_targets = self._targets_to_one_hots(targets)
        for e in range(epochs):
            print("EPOCH %i" % e)
            """
            SGD - randomize the order of the training samples, and 
            """
            in_out = list(zip(inputs, one_hot_targets))
            random.shuffle(in_out)
            # For tracking average loss over SGD, just for logging
            losses = []
            
            for inp, target in in_out:
                if inp == [-1, -1]:
                    print("TARGET: ")
                    print(target)
                    self.forward(inp)
                    losses.append(self.backward(target))

            print("LOSS: %2f" % (sum(losses)/len(losses)))
            
"""
Note that the 4th param, the size of the output layer, should be the
number of classes
"""
#MLP = NN(2, 1, 2, 2)
#xor_inputs = [[1, 1], [-1, 1], [-1, -1], [1, -1]]
#xor_labels = ([1, 0, 1, 0])
#MLP.train(xor_inputs, xor_labels)
"""
class Node(Object):
    def __init__(self):
        
    def 
"""

'\nclass Node(Object):\n    def __init__(self):\n        \n    def \n'

In [187]:
class SigmoidNN(NN):
    def _activate(self, x):
        """
        override RelU with sigmoid
        """
        return self._sigmoid(x)
    
    def _derivative_activation(self, x):
        """
        Override RelU' with Sigmoid'
        """
        return self._derivative_sigmoid(x)
    
"""
Run the same test with sigmoid
"""
MLP = SigmoidNN(2, 1, 2, 2)
xor_inputs = [[1, 1], [-1, 1], [-1, -1], [1, -1]]
xor_labels = ([1, 0, 1, 0])
MLP.train(xor_inputs, xor_labels, epochs=50, lr=.09)

EPOCH 0
TARGET: 
[ 0.  1.]
WEIGHTS:
[[ 0.12647828  1.54936555  0.        ]
 [ 0.96274059  0.01862787  0.        ]
 [-1.06116466  0.83846283  0.        ]]
[]
[[ 0.66192006 -1.57011581]
 [ 0.48394253  1.46526895]
 [ 0.70894469 -0.73593098]]
SOFTMAX_LAYER
[0.72509090015875832, 0.27490909984124173]
LOSS PER INSTANCE: 1.291315
DERIVATIVE LOSS: -3.637566
DERIVATIVE ACTIVATIONS:
[ 0.21830473  0.24685924]
LAYER: 0
UPDATING WEIGHTS
LOSS: 1.291315
EPOCH 1
TARGET: 
[ 0.  1.]
WEIGHTS:
[[ 0.12843979  1.54129241  0.        ]
 [ 0.9647021   0.01055472  0.        ]
 [-1.06312618  0.84653598  0.        ]]
[]
[[ 0.66354728 -1.56803507]
 [ 0.48901781  1.47175878]
 [ 0.72035065 -0.72134605]]
SOFTMAX_LAYER
[0.72323297514351326, 0.27676702485648663]
LOSS PER INSTANCE: 1.284579
DERIVATIVE LOSS: -3.613147
DERIVATIVE ACTIVATIONS:
[ 0.21733528  0.24743178]
LAYER: 0
UPDATING WEIGHTS
LOSS: 1.284579
EPOCH 2
TARGET: 
[ 0.  1.]
WEIGHTS:
[[ 0.13039479  1.53314537  0.        ]
 [ 0.9666571   0.00240769  0.        ]
 [

In [185]:
class TanHNN(NN):
    
    def _activate(self, x):
        """
        Override with TanH
        """
        return np.tanh(x)
    
    def _derivative_activation(self, x):
        """
        Override with Derivative TanH (1 - tanH squared)
        """
        return 1 - x*x
    
"""
Run the same test with sigmoid
"""
MLP = TanHNN(2, 1, 2, 2)
xor_inputs = [[1, 1], [-1, 1], [-1, -1], [1, -1]]
xor_labels = ([1, 0, 1, 0])
MLP.train(xor_inputs, xor_labels, epochs=50, lr=.09)

EPOCH 0
TARGET: 
[ 0.  1.]
WEIGHTS:
[[ 0.09773946 -0.8236054   0.        ]
 [ 0.42621244 -0.66779998  0.        ]
 [-2.14975304 -0.93959653  0.        ]]
[]
[[-0.84511763  0.27598692]
 [ 0.50696887  1.09852717]
 [-0.88857492  0.63128769]]
SOFTMAX_LAYER
[0.75973241513799716, 0.24026758486200278]
LOSS PER INSTANCE: 1.426002
DERIVATIVE LOSS: -4.162026
DERIVATIVE ACTIVATIONS:
[-1.72504215  0.75044268]
LAYER: 0
UPDATING WEIGHTS
LOSS: 1.426002
EPOCH 1
TARGET: 
[ 0.  1.]
WEIGHTS:
[[-5.33653148 -1.37781277  0.        ]
 [-5.0080585  -1.22200734  0.        ]
 [ 3.28451791 -0.38538916  0.        ]]
[]
[[-1.94922632  0.06703447]
 [ 1.06639408  1.20439833]
 [-0.03964791  0.79194703]]
SOFTMAX_LAYER
[3.7306204634069575e-13, 0.99999999999962708]
LOSS PER INSTANCE: 0.000000
DERIVATIVE LOSS: -1.000000
DERIVATIVE ACTIVATIONS:
[-586.7913026   -18.11984055]
LAYER: 0
UPDATING WEIGHTS
LOSS: 0.000000
EPOCH 2
TARGET: 
[ 0.  1.]
WEIGHTS:
[[-11159599.13033338    129141.94603528         0.        ]
 [-11159598.8

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:61: RuntimeWarning: overflow encountered in exp
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


In [23]:
import math
import pandas as pd

def neighborhood2int(n):
    """
    For mapping neighborhoods into integer labels
    """
    return 1 if n == "Blmngtn" else 0

df = pd.DataFrame.from_csv("./data/housing_date_train_2_features.csv")
housing_df = df.loc[df['Neighborhood'].isin(["BrDale", "Blmngtn"])]

# Grab X and its labels as a single matrix
dataset = housing_df[["LotArea", "SalePrice", "Neighborhood"]].as_matrix()

# 'shuffle' the matrix to randomize the position of each sample in it
np.random.shuffle(dataset)

# Not sure why shuffling changed the type, but need to 
# change the type of each scalar in X for some numpy methods to work
X = dataset[:, :2].astype('int64')
# Encode the labels
y = [neighborhood2int(s) for s in dataset[:, 2]]

# Find the length of 80% of the dataset for training data
train_length = int(X.shape[0] * 0.8)

# split 80%/20% for train and test
train_X = X[:train_length]
test_X = X[train_length:]